<a href="https://colab.research.google.com/github/Simonscp/Parques-Consola-Proyecto-Final-PC/blob/main/Codigo_Final_Grupo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import random  # importa el módulo 'random' para generar números aleatorios, utilizado en simulaciones como lanzar dados
import time  # importa el módulo 'time' para trabajar con funciones relacionadas con el tiempo, como pausar la ejecución
from typing import List, Tuple, Dict, Optional  # importa anotaciones de tipos para mayor claridad en el código
# list: para especificar listas
# tuple: para especificar tuplas
# dict: para especificar diccionarios
# optional: para indicar que un valor puede ser de un tipo específico o None
from dataclasses import dataclass

# dataclass es fundamental para el uso y la ejecución del código, al definir variables de tipo int y bool
@dataclass    # se utiliza ára simplificar la definición de clases que almanecan ciertos datos

#por ejemplo ficha
class Ficha:  # ficha representa cada ficha del juego y tiene los atributos nombrados a continuación
    jugador: int   # indica a qué jugadoe pertenece la ficha
    posicion: int    # guarda la posición de la ficha en el tablero
    en_carcel: bool = True   # indica si la ficha está en la carcel al inicio del juego
    en_casa: bool = False   # indica si la ficha llegó a la meta
    posicion_interna: int = -1    # lleva el control de la posición antes de llegar a casa

class JuegoParques:
    # constantes que representan espacios especiales (seguros y salidas) del tablero.
    SEGUROS = [5, 22, 39, 56]    # posiciones seguras donde las fichas no pueden ser eliminadas
    POSICIONES_SALIDA = [0, 17, 34, 51]     # posiciones de salida de cada jugador

    def __init__(self): # con self se hace referencia a los atributos de la misma clase a la que pertenece
        """
        Inicializa una instancia de JuegoParques
        Inicializa las fichas de cada jugador con su estado inicial

        Configura el número de jugadores, el tamaño del tablero, las fichas y otros parámetros
        necesarios para iniciar el juego
        """
        self.num_jugadores = self._obtener_numero_jugadores()  # define el numero de jugadores
        self.tamano_tablero = 68   # establece tamaño del tablero
        self.fichas_por_jugador = 4   # establece el número de fichas por jugador
        self.jugadores = list(range(self.num_jugadores))
        self.jugador_actual = 0
        self.fichas: Dict[int, List[Ficha]] = {}   # se usa para organizar las fichas de cada jugador
        self.modo_desarrollador = False   # perimte activar el modo desarrolador de ser necesario

        # controla otras variables auxiliares
        self.pares_consecutivos = 0  # cuenta cuántos pares consecutivos ha sacado el jugador y se reinicia si el jugador no saca un par
        self.ultima_ficha_movida = None  # guarda la referencia de la ficha que se movió en la última jugada, en casi de sacar 3 pares seguidos, ficha a la carcel
        self.movimientos_extra = 0   # almacena cantidad de movimientos extra obtenidos por sacar par o capturar ficha de otro jugador

        # colores de los equipos disponibles representados con emoticones
        self.colores_jugadores = ['🔵', '🔴', '🟢', '🟡']

        # inicializa todas las fichas de una vez.
        self._inicializar_fichas()

    def _obtener_numero_jugadores(self) -> int:
        """
        Solicita al usuario el número de equipos que jugarán

        El número debe estar entre 2 y 4. Continúa solicitando hasta obtener un valor válido

        Returns:
            int: El número de equipos
        """
        print("╔═════════════════════════════════════════════╗")
        print("║           Consola de Parqués UNAL           ║")   # imprime visual ordenada del titulo del proyecto
        print("╚═════════════════════════════════════════════╝")

        while True:   # inicia un bucle para solicitar el número de equipos hasta que el usuario ingrese un valor válido
            try:
                print("\nEquipos disponibles: 🔵 🔴 🟢 🟡")    # muestra los equipos disponibles, informando al usuario de las opciones
                num = int(input("Ingrese el número de equipos (2-4): ")) # solicita al usuario el número de equipos y convierte la entrada a entero

                if 2 <= num <= 4:
                    return num   # si el número ingresado está entre 2 y 4 (rango permitido), se retorna ese valor y se termina el bucle

                print("⚠️  El número debe estar entre 2 y 4.")    # si el número ingresado no está en el rango válido, se muestra un mensaje de advertencia

            except ValueError:   # captura el error en caso de que se ingrese texto en lugar de un número
                print("⚠️  Por favor, ingrese un número válido.")  # indica que se debe ingresar un número válido

    def _inicializar_fichas(self):
        """
        Inicializa las fichas de cada jugador con su estado inicial:

        - Cada jugador tiene 4 fichas
        - Todas las fichas comienzan en la cárcel
        """
        self.fichas = {jugador: [Ficha(jugador=jugador, posicion=-1)  # inicializa la ficha, asociandola al jugador
                                 # y establece una pocisión inicial que indica que la ficha sigue en la carcel

                                 for _ in range(self.fichas_por_jugador)]  # cantidad de fichas por jugador
                      for jugador in self.jugadores}

    def lanzar_dados(self) -> Tuple[int, int]:
        """
        Simula el lanzamiento de dos dados de seis caras

        En modo desarrollador, permite elegir entre valores aleatorios o introducir manualmente los resultados.
        En modo normal, muestra un efecto de lanzamiento con pausas

        Returns:
            Tuple[int, int]: Una tupla con los resultados de los dos dados
        """
        if self.modo_desarrollador:  # comprueba si el juego está en modo desarrollador para ofrecer opciones adicionales
                # solicita al usuario elegir entre un lanzamiento aleatorio (R) o manual (M)
            eleccion = input("R=aleatorio, M=manual: ").upper()   # usa upper convirtiendo la opción a mayúsculas para normalizarla
            if eleccion == 'R':
                dados = (random.randint(1, 6), random.randint(1, 6)) # genera dos números aleatorios entre 1 y 6 simulando el lanzamiento de los dados
                print(f"🎲 Dados: {dados[0]}, {dados[1]}")  # imprime el resultado de los dados
                return dados

            elif eleccion == 'M':
                try:
                    d1 = int(input("Dado 1 (1-6): "))
                    d2 = int(input("Dado 2 (1-6): "))
                    # pide al usuario ingresar manualmente el resultado de cada dado y lo convierte a entero

                    if 1 <= d1 <= 6 and 1 <= d2 <= 6:
                        return (d1, d2)   # si ambos valores están en el rango permitido (1 a 6), retorna los valores
                    print("⚠️  Los valores deben estar entre 1 y 6.")
                except ValueError:  # captura el error en caso de que se ingrese texto en lugar de un número
                    print("⚠️  Entrada inválida.")
                return self.lanzar_dados()
            else:
                return self.lanzar_dados()

        print("Lanzando dados", end="")  # en modo normal, muestra el mensaje "Lanzando dados" sin salto de línea
        for _ in range(3):
            print(".", end="", flush=True)
            time.sleep(0.2)  # imprime tres puntos consecutivos en la misma línea, con una pausa de 0.2 segundos entre cada uno
            # simula el efecto visual de dados siendo lanzados
        print()  # imprime un salto de línea para separar la animación del resultado final

        dados = (random.randint(1, 6), random.randint(1, 6))  # genera dos números aleatorios entre 1 y 6 simulando el lanzamiento de los dados
        print(f"🎲 Dados: {dados[0]}, {dados[1]}")  # imprime el resultado final de los dados usando un emoji para mayor claridad visual
        return dados

    def es_seguro(self, posicion: int) -> bool:
        """
        Determina si una posición en el tablero es segura

        Una posición es segura si se encuentra en la lista de casillas seguras

        Args:
            posicion (int): La posición a verificar

        Returns:
            bool: True si la posición es segura, False en caso contrario
        """
         # comprueba si la posición indicada está dentro de la lista de posiciones seguras
        return posicion in self.SEGUROS

    def obtener_fichas_en_posicion(self, posicion: int) -> List[Ficha]:
        """
        Obtiene todas las fichas que se encuentran en una posición específica del tablero

        Args:
            posicion (int): La posición en el tablero

        Returns:
            List[Ficha]: Una lista de fichas que se encuentran en esa posición
        """
         # crea una lista vacía para almacenar las fichas que se encuentren en la posición dada
        result = []
         # itera sobre todas las fichas de todos los jugadores

        for fichas_jugador in self.fichas.values():  # recorre cada ficha dentro de la lista del jugador

            for ficha in fichas_jugador:   # verifica tres condiciones
                if (ficha.posicion == posicion and  # la ficha se encuentra en la posición indicada
                    not ficha.en_carcel and  # la ficha no está en la cárcel (en_carcel es False)
                    not ficha.en_casa):  # la ficha no ha llegado a casa (en_casa es False)
                    # si todas las condiciones se cumplen, se añade la ficha a la lista result
                    result.append(ficha)

        return result  # retorna la lista con todas las fichas que cumplen las condiciones.

    def hay_bloqueo_antes(self, posicion_inicio: int, posicion_fin: int) -> bool:
        """
        Verifica si existe un bloqueo en el camino de una ficha entre dos posiciones

        Un bloqueo ocurre si hay dos fichas del mismo jugador en cualquier casilla entre la posición de inicio y la posición de destino.

        Args:
            posicion_inicio (int): La posición inicial
            posicion_fin (int): La posición destino

        Returns:
            bool: True si hay un bloqueo, False en caso contrario.
        """
        if posicion_fin < posicion_inicio:
            posicion_fin += self.tamano_tablero

        for pos in range(posicion_inicio + 1, posicion_fin):
            pos_actual = pos % self.tamano_tablero
            fichas = self.obtener_fichas_en_posicion(pos_actual)
            if len(fichas) == 2 and fichas[0].jugador == fichas[1].jugador:
                return True
        return False

    def puede_mover_ficha(self, ficha: Ficha, espacios: int) -> bool:
        """
        Verifica si una ficha puede moverse una cantidad de espacios dada

        Considera si la ficha está en casa, en cárcel, si hay bloqueos en el trayecto y si el destino es válido.

        Args:
            ficha (Ficha): La ficha a mover
            espacios (int): La cantidad de espacios para mover

        Returns:
            bool: True si el movimiento es válido, False en caso contrario
        """
        # Si la ficha aún está en casa, no puede moverse al tablero
        if ficha.en_casa:
            return False
        # Si la ficha está en la cárcel, solo se puede mover si se obtiene el 5
        if ficha.en_carcel:
            return espacios == 5

        # Guarda la posición actual de la ficha
        posicion_actual = ficha.posicion
        # Calcula la nueva posición usando el módulo para que no se salga del tamaño del tablero
        nueva_posicion = (posicion_actual + espacios) % self.tamano_tablero

        # Verifica si hay algún bloqueo en el camino entre la posición actual y la nueva posición
        if self.hay_bloqueo_antes(posicion_actual, nueva_posicion):
            return False

        # Obtiene todas las fichas que se encuentran en la nueva posición destino
        fichas_en_destino = self.obtener_fichas_en_posicion(nueva_posicion)
        # Si en la posición destino hay exactamente dos fichas:
        if len(fichas_en_destino) == 2:
           # Si ambas fichas pertenecen al mismo jugador, el movimiento no es válido
            if fichas_en_destino[0].jugador == fichas_en_destino[1].jugador:
                return False
            # Además, si la posición es segura, no se permite mover
            if self.es_seguro(nueva_posicion):
                return False
      # Si pasa todos los requerimientos, el movimiento es válido
        return True

    def mover_ficha(self, ficha: Ficha, espacios: int) -> bool:
        """
        Mueve una ficha la cantidad de espacios especificada, actualizando su estado en el juego

        Si la ficha está en la cárcel y el movimiento es válido, la saca de la cárcel y la mueve a la salida
        También gestiona capturas de fichas de otros jugadores y asigna movimientos extra en caso de captura

        Args:
            ficha (Ficha): La ficha a mover.
            espacios (int): La cantidad de espacios para mover.

        Retorno:
            bool: True si el movimiento se realizó correctamente, False en caso contrario.
        """
        # Primero, se verifica que el movimiento sea válido
        if not self.puede_mover_ficha(ficha, espacios):
            return False

        # Para el caso especial de salir de la cárcel
        if ficha.en_carcel and espacios == 5:
          # Saca la ficha de la cárcel
            ficha.en_carcel = False
          # Coloca la ficha en la posición de salida asignada a su jugador
            ficha.posicion = self.POSICIONES_SALIDA[ficha.jugador]
          # Registra esta ficha como la última que se movió
            self.ultima_ficha_movida = ficha
            return True

        # Se calcula la nueva posición de la ficha en el tablero
        nueva_posicion = (ficha.posicion + espacios) % self.tamano_tablero
        # Obtiene todas las fichas que se encuentran en la nueva posición destino
        fichas_en_objetivo = self.obtener_fichas_en_posicion(nueva_posicion)

        # Aqui se inica la variable para saber si se realiza una captura
        captura_realizada = False
        # Recorre cada ficha que se encuentre en la posición destino
        for ficha_objetivo in fichas_en_objetivo:
           # Si la ficha en destino pertenece a otro jugador y la posición no es segura
            if (ficha_objetivo.jugador != ficha.jugador and
                not self.es_seguro(nueva_posicion)):
                # si existe captura: la ficha se manda a la cárcel
                ficha_objetivo.en_carcel = True
                # Se asigna una posición inválida, en este caso -1 para indicar que no está en el tablero
                ficha_objetivo.posicion = -1
                 # Marca que se ha realizado una captura
                captura_realizada = True
        # Actualiza la posición de la ficha que se mueve
        ficha.posicion = nueva_posicion
        # Registra la ficha como la última movida
        self.ultima_ficha_movida = ficha
       # Si se realizó alguna captura, se recompensa al jugador con 20 movimientos extra
        if captura_realizada:
            self.movimientos_extra += 20
            print("🏆 ¡Captura! +20 movimientos extra")

      # Si el movimiento se completó exitosamente
        return True

    def mostrar_tablero(self):
        """
        Limpia la consola y muestra el estado actual del tablero.

        Incluye una cabecera con información del turno actual, las fichas en cárcel y en casa, y
        una representación visual simplificada del tablero, marcando casillas especiales y la posición de las fichas.
        """
          # Se muestra el turno actual y el color del jugador que juega
        print("╔═══════════════════════════════╗")
        print(f"║      PARQUÉS - TURNO {self.jugador_actual + 1} {self.colores_jugadores[self.jugador_actual]}      ║")
        print("╚═══════════════════════════════╝")
        # Si existen movimientos extra disponibles, se muestran
        if self.movimientos_extra > 0:
            print(f"🎁 Bonus: {self.movimientos_extra} movimientos extra")

        # Muestra el estado actual del juego (cantidad de fichas en cárcel y en casa por jugador)
        print("\n📊 ESTADO DEL JUEGO:")
        print("┌─────────┬───────┬───────┐")
        print("│ Jugador │ Cárcel│  Casa │")
        print("├─────────┼───────┼───────┤")

        # Para cada jugador, calcula cuántas fichas están en cárcel y cuántas en casa
        for jugador in self.jugadores:
            fichas_carcel = sum(1 for f in self.fichas[jugador] if f.en_carcel)
            fichas_casa = sum(1 for f in self.fichas[jugador] if f.en_casa)
            color = self.colores_jugadores[jugador]
            print(f"│   {jugador+1} {color}   │   {fichas_carcel}   │   {fichas_casa}   │")
        print("└─────────┴───────┴───────┘")
        # Imprime el encabezado del tablero de juego
        print("\n🎮 TABLERO:")

       # Se crea un conjunto para almacenar solo las posiciones que serán mostradas
        posiciones_mostradas = set()
       # Agrega las posiciones especiales: zonas seguras y de salida
        for pos in self.SEGUROS + self.POSICIONES_SALIDA:
            posiciones_mostradas.add(pos)
        # Agrega las posiciones donde hay fichas en juego, excluyendo las fichas en casa o en cárcel
        for jugador in self.jugadores:
            for ficha in self.fichas[jugador]:
                if not ficha.en_carcel and not ficha.en_casa:
                    posiciones_mostradas.add(ficha.posicion)
        # Recorre y muestra las posiciones ordenadas
        for pos in sorted(posiciones_mostradas):
           # Inicia la línea con el número de posición actualizadas
            linea = f"[{pos:2d}] "
             # Indica si la posición es especial (zona segura o de salida)
            if pos in self.SEGUROS:
                linea += "🛡️ "
            elif pos in self.POSICIONES_SALIDA:
                idx = self.POSICIONES_SALIDA.index(pos)
                linea += f"🚪{idx+1} "
            else:
                linea += "   "
             # Obtiene las fichas en la posición actual y las muestra con su color correspondiente
            fichas = self.obtener_fichas_en_posicion(pos)
            for ficha in fichas:
                linea += f"{self.colores_jugadores[ficha.jugador]} "
            # Imprime la línea con la información de la posición y las fichas presentes
            print(linea)

    def seleccionar_ficha(self, fichas_validas: List[Ficha]) -> Optional[Ficha]:
        """
        Permite al usuario seleccionar una ficha válida para mover.

        Muestra las fichas disponibles para el jugador actual y solicita una selección. Si no se selecciona
        ninguna ficha, retorna None.

        Args:
            fichas_validas (List[Ficha]): Lista de fichas que pueden moverse.

        Returns:
            Optional[Ficha]: La ficha seleccionada o None si no se seleccionó ninguna.
        """
        # Si no hay fichas válidas, se notifica y se retorna None
        if not fichas_validas:
            print("❌ No hay movimientos válidos")
            time.sleep(1)
            return None
        # Se muestran las fichas disponibles
        print("\n📌 Fichas disponibles:")
        # Recorre las fichas válidas y muestra su índice, color y estado.
        for i, ficha in enumerate(fichas_validas):
            color = self.colores_jugadores[ficha.jugador]
            if ficha.en_carcel:
                estado = "en cárcel"
            else:
                estado = f"en posición {ficha.posicion}"
            print(f"  {i}: {color} Ficha {estado}")

        try:
          # Solicita al usuario que ingrese el índice de la ficha a mover o que pulse Enter para saltar
            opcion = input("\nSeleccione ficha (número) o pulse Enter para saltar: ")
             # Si el usuario pulsa Enter sin ingresar nada, no se selecciona ninguna ficha
            if not opcion:
                return None

          # Se convierte la opción ingresada a entero
            eleccion = int(opcion)
            # Verifica que la opción esté dentro del rango válido
            if 0 <= eleccion < len(fichas_validas):
                return fichas_validas[eleccion]
            else:
                print("⚠️  Selección inválida")
                time.sleep(1)
        except ValueError:
            # Si la entrada no es un número, notifica el error
            print("⚠️  Entrada inválida")
            time.sleep(1)

        return None


    def jugar_turno(self):
        """
        Ejecuta el turno del jugador actual.

        Muestra el tablero y anuncia el turno del jugador actual. Luego, lanza los dados y evalúa
        si se obtienen pares (dados iguales). Si se logran tres pares consecutivos, se penaliza al
        jugador enviando la última ficha movida a la cárcel. Finalmente, procesa los movimientos
        basados en los valores de los dados y, si no se obtuvo un par, cambia el turno al siguiente jugador.
        """
        self.mostrar_tablero()  # Muestra el estado actual del tablero
        print(f"\n🎯 Turno del Jugador {self.jugador_actual + 1} {self.colores_jugadores[self.jugador_actual]}")
        dados = self.lanzar_dados()  # Lanza los dados y obtiene los valores
        suma_dados = dados[0] + dados[1]  # Calcula la suma de los valores de los dados

        if dados[0] == dados[1]:  # Verifica si los dados son iguales (par)
            self.pares_consecutivos += 1  # Incrementa el contador de pares consecutivos
            print(f"🎯 ¡Par! ({self.pares_consecutivos} consecutivos)")

            if self.pares_consecutivos == 3:  # Si hay tres pares consecutivos, se penaliza al jugador
                if self.ultima_ficha_movida:  # Verifica si hay una última ficha movida
                    self.ultima_ficha_movida.en_carcel = True  # Envía la ficha a la cárcel
                    self.ultima_ficha_movida.posicion = -1  # Reinicia la posición de la ficha
                    print("⚠️ ¡Tres pares seguidos! Última ficha movida vuelve a la cárcel.")
                    time.sleep(2)  # Pausa para mostrar el mensaje
                self.pares_consecutivos = 0  # Reinicia el contador de pares consecutivos
                return  # Termina el turno inmediatamente
        else:
            self.pares_consecutivos = 0  # Reinicia el contador si no se obtuvo un par

        self._procesar_movimientos_turno(dados)  # Procesa los movimientos del jugador según los dados

        if dados[0] != dados[1]:  # Si no obtuvo un par, cambia el turno al siguiente jugador
            self.jugador_actual = (self.jugador_actual + 1) % self.num_jugadores

    def _procesar_movimientos_turno(self, dados: Tuple[int, int]):
        """
        Procesa los movimientos del turno basado en el resultado de los dados.
        """
        self._usar_movimientos_extra()  # Usa movimientos extra si los hay
        movimientos = list(dados)  # Convierte los valores de los dados en una lista de movimientos

        while movimientos:  # Itera sobre los movimientos disponibles
            self.mostrar_tablero()  # Muestra el tablero antes de cada movimiento
            print(f"🎲 Movimiento disponible: {movimientos[0]}")

            if not self._realizar_movimiento(movimientos[0]):  # Intenta mover una ficha con el valor del dado
                movimientos.pop(0)  # Si no se puede mover, elimina ese valor de la lista
                continue  # Continúa con el siguiente movimiento

            movimientos.pop(0)  # Elimina el movimiento después de usarlo

        if len(movimientos) == 2 and movimientos[0] == movimientos[1]:  # Si los dados son iguales
            self.mostrar_tablero()
            print(f"🎲 Puedes usar la suma: {sum(movimientos)}")
            self._realizar_movimiento(sum(movimientos))  # Permite mover con la suma de los valores

    def _usar_movimientos_extra(self):
        """
        Ejecuta los movimientos extra acumulados durante el turno.
        """
        while self.movimientos_extra > 0:  # Mientras haya movimientos extra
            self.mostrar_tablero()
            print(f"🎁 Movimientos extra: {self.movimientos_extra}")

            movimiento = min(self.movimientos_extra, 6)  # Determina el movimiento a realizar (máximo 6)

            if not self._realizar_movimiento(movimiento):  # Intenta realizar el movimiento extra
                if movimiento > 1:
                    self.movimientos_extra -= 1  # Reduce los movimientos extra si no se pudo mover
                    continue
                else:
                    break  # Si no se puede hacer ningún movimiento, termina el bucle

            self.movimientos_extra = max(0, self.movimientos_extra - movimiento)  # Reduce el total de movimientos extra

    def _realizar_movimiento(self, espacios: int) -> bool:
        """
        Intenta realizar un movimiento para la ficha seleccionada.
        """
        fichas_validas = [f for f in self.fichas[self.jugador_actual]
                          if self.puede_mover_ficha(f, espacios)]  # Obtiene las fichas que pueden moverse
        ficha = self.seleccionar_ficha(fichas_validas)  # Permite seleccionar una ficha válida

        if not ficha:
            return False  # Si no hay fichas válidas, retorna False

        if self.mover_ficha(ficha, espacios):  # Intenta mover la ficha seleccionada
            print(f"✅ Ficha movida a posición {ficha.posicion}")
            time.sleep(1)  # Pausa breve para visualizar el movimiento
            return True  # Retorna True si el movimiento fue exitoso

        return False  # Retorna False si el movimiento no se pudo realizar

    def jugar_partida(self):
        """
        Inicia y gestiona el desarrollo completo de la partida.
        """
        modo = input("Modo: [1] Normal, [2] Desarrollador: ")  # Pide al usuario elegir el modo de juego
        self.modo_desarrollador = modo == "2"  # Activa el modo desarrollador si elige la opción 2

        while True:  # Bucle principal del juego
            ganador = self.verificar_ganador()  # Verifica si hay un ganador

            if ganador is not None:  # Si hay un ganador, muestra el mensaje y termina la partida
                self.mostrar_tablero()
                print(f"\n🏆 ¡El Jugador {ganador + 1} {self.colores_jugadores[ganador]} gana!")
                break

            self.jugar_turno()  # Ejecuta el turno del jugador actual
            input("\nPresione Enter para continuar...")  # Espera la interacción del usuario

    def verificar_ganador(self) -> Optional[int]:
        """
        Verifica si algún jugador ha ganado la partida.
        """
        for jugador in self.jugadores:
            if all(ficha.en_casa for ficha in self.fichas[jugador]):  # Verifica si todas las fichas del jugador están en casa
                return jugador  # Retorna el índice del jugador ganador

        return None  # Retorna None si no hay ganador aún

# Crea y ejecuta el juego
if __name__ == "__main__":
    juego = JuegoParques()  # Crea una instancia del juego
    juego.jugar_partida()  # Inicia la partida



╔═════════════════════════════════════════════╗
║           Consola de Parqués UNAL           ║
╚═════════════════════════════════════════════╝

Equipos disponibles: 🔵 🔴 🟢 🟡
